<a href="https://colab.research.google.com/github/ElizavetaNosova/Ordering-text-quest-fragments/blob/main/gpt3_ordering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import os
import json
import math
from networkx.readwrite import json_graph
from tqdm import tqdm
import random
from copy import copy

In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
os.chdir('gdrive/MyDrive')

In [ ]:
MODEL_PATH = "home/jovyan/devices/quests/hf"
TOKENIZER_PATH = "home/jovyan/devices/quests/hf"

In [ ]:
gpt_model = GPT2LMHeadModel.from_pretrained(MODEL_PATH)
tokenizer = GPT2Tokenizer.from_pretrained(TOKENIZER_PATH)

In [ ]:
gpt_model.cuda()
gpt_model.eval()

In [ ]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [ ]:
GPT3_LIMIT = 2048
FULL_LIMIT = 1500
def calc_perplexity_based_score(history_tokens, next_candidate_tokens, candidate_limit=500): 
    history_limit = FULL_LIMIT-min(candidate_limit, len(next_candidate_tokens))
    sequence_tokens = history_tokens[-history_limit:] + next_candidate_tokens[:candidate_limit]

    input_idx = torch.tensor([sequence_tokens])
    input_idx = input_idx.cuda()
    with torch.no_grad():
        loss = float(gpt_model(input_idx, labels=input_idx)[0].cpu())
    return 1-sigmoid(loss)

In [ ]:
start_token = 'BOQ'

class Fragment:
    bof_token = 'BOF'
    button_token = 'BUTTON'
    input_token = 'INPUT'
    default_input='Далее'

    def __init__(self, fragment_id, graph=None, full_text = None):
        self.fragment_id = fragment_id
        if graph:
            fragment_text = graph.nodes()[fragment_id]['fragment_text']
            edges = graph.edges()
            buttons =  [' '.join([self.button_token, edges[edge]['command']] + [self.default_input]) for edge in graph.out_edges(fragment_id)]
            full_text = ' '.join([self.bof_token, fragment_text]) 
            if buttons:
                full_text = full_text + ' '.join(buttons)
        full_text = ' '.join([full_text, self.input_token, self.default_input])
        self.tokens = tokenizer.encode(full_text)

In [ ]:
NEW_GRAPHS_DIRECTORY = 'new_questbook_graphs'
NEW_PATHS_DIRECTORY = 'new_questbook_paths'

In [ ]:
def gpt_beam_search(test_sample, quest_graph, num_best_candidates=2):
    fragments = [Fragment(node_id, quest_graph) for node_id in test_sample]
    
    order_candidates = [{'beginning':[], 'next_candidates': fragments, 'probability':1, 'history':tokenizer.encode(start_token)}]
    random.shuffle(order_candidates[0]['next_candidates'])
    
    for i in range(len(fragments)):
        current_iteration_next_candidates = []
        
        for order_candidate in order_candidates:
            history = order_candidate ['history']
            
            
            scored_next = [{'next_fragment':next_fragment, 'score':calc_perplexity_based_score(history, next_fragment.tokens)} for next_fragment in order_candidate['next_candidates']]
            scored_next.sort(key=lambda x: x['score'], reverse=True)
            
            chosen_next_candidates = scored_next[:num_best_candidates] if len(scored_next) < num_best_candidates else scored_next
            for chosen_next_candidate in chosen_next_candidates:
                
                next_step_beginning = copy(order_candidate['beginning'])
                next_step_beginning.append(chosen_next_candidate['next_fragment'])
                
                next_step_probability = order_candidate['probability']*chosen_next_candidate['score']
                
                next_step_next_candidates = []
                for next_step_next_candidate in order_candidate['next_candidates']:
                    if next_step_next_candidate.fragment_id != chosen_next_candidate['next_fragment'].fragment_id:
                        next_step_next_candidates.append(next_step_next_candidate)
                next_step_history = history + chosen_next_candidate['next_fragment'].tokens
                
              
                current_iteration_next_candidates.append({'beginning':next_step_beginning, 'next_candidates':next_step_next_candidates, 'probability':next_step_probability, 'history': next_step_history})
        
        current_iteration_next_candidates.sort(key=lambda x: x['probability'], reverse=True)
       # print(len(current_iteration_next_candidates))
        if current_iteration_next_candidates:
            order_candidates = current_iteration_next_candidates[:num_best_candidates] if len(current_iteration_next_candidates) > num_best_candidates else  current_iteration_next_candidates
    return [fragment.fragment_id for fragment in order_candidates[0]['beginning']]

In [ ]:
results = []
fails = []
for file_name in tqdm(os.listdir(NEW_GRAPHS_DIRECTORY)[3:]):
    used_current_file_paths = 0
    graph_file = os.path.join(NEW_GRAPHS_DIRECTORY, file_name)
    with open(graph_file) as f:
        G = json_graph.node_link_graph(json.load(f))
        nodes = G.nodes()

    paths_file = os.path.join(NEW_PATHS_DIRECTORY, file_name)
    with open(paths_file) as f:
        paths = json.load(f)
    
    for path in paths:
        corrected_path = [node_id for node_id in path if 'fragment_text' in nodes[node_id] and isinstance(nodes[node_id]['fragment_text'], str)]
        if len(corrected_path) > 3 and len(corrected_path) <= 25:
            try:
                predicted = gpt_beam_search(corrected_path, G)

                results.append({'correct':path, 'predicted':predicted})
                used_current_file_paths += 1
                if len(results) % 10 == 0:
                    with open('gpt3_ordering_1500.json', 'w') as f:
                        json.dump(results, f)
                if used_current_file_paths == 10:
                    break
            except Exception as e:
                print(e)
                
with open('gpt3_ordering_1500.json', 'w') as f:
    json.dump(results, f)

In [ ]:
# The first sequences were generated without the last iteration by mistake
def fix_prediction(predicted, correct):
    if len(predicted) == len(correct):
        return predicted
    elif len(predicted) < len(correct):
        unordered_items = [item for item in correct if item not in predicted]
        random.shuffle(unordered_items)
        return predicted + unordered_items
    else:
        raise ValueError('wrong sequence length')

In [ ]:
import pandas as pd
from scipy.stats import kendalltau

def longest_correct_subsequence(predicted, correct):
    correct_transitions = set([(item_from, item_to) for item_from, item_to in zip(correct[:-1], correct[1:])])
    predicted_transitions = [(item_from, item_to) for item_from, item_to in zip(predicted[:-1], predicted[1:])]
    predicted_transitions_are_correct = [transition in correct_transitions for transition in predicted_transitions]
    
    longest_correct_transitions_subsequence = 0
    current_correct_transitions_subsequence = 0
    #Add False as last item to include last real item checking into the loop
    for predicted_transition_is_correct in predicted_transitions_are_correct + [False]:
        if predicted_transition_is_correct:
            current_correct_transitions_subsequence += 1
        else:
            if current_correct_transitions_subsequence > longest_correct_transitions_subsequence:
                longest_correct_transitions_subsequence = current_correct_transitions_subsequence
            current_correct_transitions_subsequence = 0
    #return number of items in longest correct sequence (not number of transitions)
    return longest_correct_transitions_subsequence + 1 if longest_correct_transitions_subsequence else 0  

In [ ]:
gpt_1500_df = pd.DataFrame(columns=['Fragment_length', 'Tau', 'Max correct sequence'])
exceptions = []

with open('gpt3_ordering_1500.json') as f:
   results = json.load(f)

for sample in results:
    correct = sample['correct']
    try:
        predicted = fix_prediction(sample['predicted'], correct)
        tau = kendalltau(predicted, correct).correlation
        lcs = longest_correct_subsequence(predicted, correct)
        gpt_1500_df.loc[len(gpt_1500_df)] = [len(correct), tau, lcs]
    except:
        exceptions.append(sample)

In [ ]:
gpt_1500_df.groupby('Fragment_length').describe().keys()

MultiIndex([(                 'Tau', 'count'),
            (                 'Tau',  'mean'),
            (                 'Tau',   'std'),
            (                 'Tau',   'min'),
            (                 'Tau',   '25%'),
            (                 'Tau',   '50%'),
            (                 'Tau',   '75%'),
            (                 'Tau',   'max'),
            ('Max correct sequence', 'count'),
            ('Max correct sequence',  'mean'),
            ('Max correct sequence',   'std'),
            ('Max correct sequence',   'min'),
            ('Max correct sequence',   '25%'),
            ('Max correct sequence',   '50%'),
            ('Max correct sequence',   '75%'),
            ('Max correct sequence',   'max')],
           )

In [ ]:
aggr_gpt1500 = gpt_1500_df.groupby('Fragment_length').describe()[[('Tau',  'count'), ('Tau',  'mean'),  ('Max correct sequence',  'mean')]]
aggr_gpt1500

Tau           Max correct sequence
                count      mean                 mean
Fragment_length                                     
4.0               3.0  0.111111             3.000000
5.0              15.0  0.013333             0.866667
6.0               7.0 -0.161905             1.857143
7.0               6.0 -0.238095             2.666667
8.0               9.0 -0.079365             2.444444
9.0              16.0 -0.006944             2.125000
10.0             27.0  0.142387             2.074074
11.0             19.0  0.071770             2.105263
12.0             32.0  0.011364             1.687500
13.0             25.0  0.074872             2.240000
14.0             13.0 -0.085376             2.076923
15.0             11.0 -0.075325             2.363636
16.0              7.0 -0.016667             1.857143
17.0              6.0  0.009804             2.000000
18.0             11.0  0.100416             2.090909
19.0              6.0  0.056530             1.000000
20.0              5.0  0.077895             1.600000
21.0              1.0  0.047619             3.000000
22.0              2.0 -0.173160             2.000000
23.0              2.0 -0.055336             2.000000
24.0              2.0  0.123188             1.000000
25.0             10.0  0.119333             1.900000
26.0              2.0 -0.141538             2.500000

In [ ]:
with open('latex.txt', 'w') as f:
  f.write(aggr_gpt1500.to_latex())

In [ ]:
len(sample['correct'])

20

In [ ]:
len(sample['predicted'])

18

In [ ]:
FULL_LIMIT = GPT3_LIMIT

results = []
fails = []
for file_name in tqdm(os.listdir(NEW_GRAPHS_DIRECTORY)):
    used_current_file_paths = 0
    graph_file = os.path.join(NEW_GRAPHS_DIRECTORY, file_name)
    with open(graph_file) as f:
        G = json_graph.node_link_graph(json.load(f))
        nodes = G.nodes()

    paths_file = os.path.join(NEW_PATHS_DIRECTORY, file_name)
    with open(paths_file) as f:
        paths = json.load(f)
    
    for path in paths:
        corrected_path = [node_id for node_id in path if 'fragment_text' in nodes[node_id] and isinstance(nodes[node_id]['fragment_text'], str)]
        if len(corrected_path) > 3 and len(corrected_path) <= 25:
            try:
                predicted = gpt_beam_search(corrected_path, G)

                results.append({'correct':path, 'predicted':predicted})
                used_current_file_paths += 1
                if len(results) % 10 == 0:
                    with open('gpt3_ordering_2048.json', 'w') as f:
                        json.dump(results, f)
                if used_current_file_paths == 10:
                    break
            except Exception as e:
                fails.append[(file_name, e)]
                
with open('gpt3_ordering_2048.json', 'w') as f:
    json.dump(results, f)


100%|██████████| 34/34 [2:39:50<00:00, 282.08s/it]


In [ ]:
gpt_2048_df = pd.DataFrame(columns=['Fragment_length', 'Tau', 'Max correct sequence'])
exceptions = []

with open('gpt3_ordering_2048.json') as f:
   results = json.load(f)

for sample in results:
    correct = sample['correct']
    try:
        predicted = fix_prediction(sample['predicted'], correct)
        tau = kendalltau(predicted, correct).correlation
        lcs = longest_correct_subsequence(predicted, correct)
        gpt_2048_df.loc[len(gpt_2048_df)] = [len(correct), tau, lcs]
    except:
        exceptions.append(sample)

In [ ]:
aggr_gpt2048 = gpt_2048_df.groupby('Fragment_length').describe()[[('Tau',  'count'), ('Tau',  'mean'),  ('Max correct sequence',  'mean')]]
aggr_gpt2048

Tau               Max correct sequence
                count          mean                 mean
Fragment_length                                         
4.0               3.0 -1.110223e-16             2.333333
5.0              15.0  1.066667e-01             1.200000
6.0               7.0 -6.666667e-02             2.142857
7.0               6.0 -2.698413e-01             2.666667
8.0               9.0 -8.730159e-02             2.666667
9.0              16.0 -2.083333e-02             2.000000
10.0             23.0  1.516908e-01             2.217391
11.0             19.0  9.473684e-02             2.210526
12.0             32.0  1.325758e-02             1.750000
13.0             21.0  5.006105e-02             2.523810
14.0             13.0 -4.142012e-02             2.153846
15.0             11.0 -9.523810e-03             2.000000
16.0              7.0 -7.142857e-03             1.571429
17.0              6.0 -4.625929e-18             1.500000
18.0              8.0  1.225490e-01             1.875000
19.0              6.0 -8.382066e-02             1.833333
20.0              5.0  6.315789e-03             2.000000
21.0              1.0 -3.809524e-02             3.000000
22.0              2.0  4.761905e-02             1.000000
23.0              2.0  4.347826e-02             0.000000
24.0              2.0  6.521739e-02             1.000000
25.0              5.0 -6.000000e-02             2.400000
26.0              2.0 -6.769231e-02             1.000000

In [ ]:
with open('latex.txt', 'w') as f:
  f.write(aggr_gpt1500.to_latex())

In [ ]:
def gpt_beam_search_last_score(test_sample, quest_graph, num_best_candidates=2):
    fragments = [Fragment(node_id, quest_graph) for node_id in test_sample]
    
    order_candidates = [{'beginning':[], 'next_candidates': fragments, 'probability':1, 'history':tokenizer.encode(start_token)}]
    random.shuffle(order_candidates[0]['next_candidates'])
    
    for i in range(len(fragments)):
        current_iteration_next_candidates = []
        
        for order_candidate in order_candidates:
            history = order_candidate ['history']
            
            
            scored_next = [{'next_fragment':next_fragment, 'score':calc_perplexity_based_score(history, next_fragment.tokens)} for next_fragment in order_candidate['next_candidates']]
            scored_next.sort(key=lambda x: x['score'], reverse=True)
            
            chosen_next_candidates = scored_next[:num_best_candidates] if len(scored_next) < num_best_candidates else scored_next
            for chosen_next_candidate in chosen_next_candidates:
                
                next_step_beginning = copy(order_candidate['beginning'])
                next_step_beginning.append(chosen_next_candidate['next_fragment'])
                
                next_step_probability = chosen_next_candidate['score']
                
                next_step_next_candidates = []
                for next_step_next_candidate in order_candidate['next_candidates']:
                    if next_step_next_candidate.fragment_id != chosen_next_candidate['next_fragment'].fragment_id:
                        next_step_next_candidates.append(next_step_next_candidate)
                next_step_history = history + chosen_next_candidate['next_fragment'].tokens
                
              
                current_iteration_next_candidates.append({'beginning':next_step_beginning, 'next_candidates':next_step_next_candidates, 'probability':next_step_probability, 'history': next_step_history})
        
        current_iteration_next_candidates.sort(key=lambda x: x['probability'], reverse=True)
       # print(len(current_iteration_next_candidates))
        if current_iteration_next_candidates:
            order_candidates = current_iteration_next_candidates[:num_best_candidates] if len(current_iteration_next_candidates) > num_best_candidates else  current_iteration_next_candidates
    return [fragment.fragment_id for fragment in order_candidates[0]['beginning']]

In [ ]:
import math
FULL_LIMIT = GPT3_LIMIT
def scoring_gpt_beam_search_last_score(test_sample,  num_best_candidates=2, epsilon=10**(-9)):
    fragments = [Fragment(i, full_text = text) for i, text in enumerate(test_sample)]
    
    order_candidates = [{'beginning':[], 'next_candidates': fragments, 'probability':1, 'history':tokenizer.encode(start_token), 'aggregated_score': 0}]
    random.shuffle(order_candidates[0]['next_candidates'])
    
    for i in range(len(fragments)):
        current_iteration_next_candidates = []
        
        for order_candidate in order_candidates:
            history = order_candidate ['history']
            
            
            scored_next = [{'next_fragment':next_fragment, 'score':calc_perplexity_based_score(history, next_fragment.tokens)} for next_fragment in order_candidate['next_candidates']]
            scored_next.sort(key=lambda x: x['score'], reverse=True)
            
            chosen_next_candidates = scored_next[:num_best_candidates] if len(scored_next) < num_best_candidates else scored_next
            for chosen_next_candidate in chosen_next_candidates:
                
                next_step_beginning = copy(order_candidate['beginning'])
                next_step_beginning.append(chosen_next_candidate['next_fragment'])
                
                next_step_probability = chosen_next_candidate['score']
                
                next_step_next_candidates = []
                for next_step_next_candidate in order_candidate['next_candidates']:
                    if next_step_next_candidate.fragment_id != chosen_next_candidate['next_fragment'].fragment_id:
                        next_step_next_candidates.append(next_step_next_candidate)
                next_step_history = history + chosen_next_candidate['next_fragment'].tokens
                
              
                current_iteration_next_candidates.append({'beginning':next_step_beginning, 'next_candidates':next_step_next_candidates, 'probability':next_step_probability, 'history': next_step_history, 'aggregated_score': order_candidate['probability']+math.log(max(next_step_probability, epsilon))})
        
        current_iteration_next_candidates.sort(key=lambda x: x['probability'], reverse=True)
       # print(len(current_iteration_next_candidates))
        if current_iteration_next_candidates:
            order_candidates = current_iteration_next_candidates[:num_best_candidates] if len(current_iteration_next_candidates) > num_best_candidates else  current_iteration_next_candidates
    return {'order': [fragment.fragment_id for fragment in order_candidates[0]['beginning']], 'score': order_candidates[0]['aggregated_score']}

In [ ]:
with open('sanity_check_data.json') as f:
    sanity_check_data = json.load(f)

In [ ]:
real_quests_predictions = [scoring_gpt_beam_search_last_score(path) for path in sanity_check_data['real']]
random_quests_predictions = [scoring_gpt_beam_search_last_score(path) for path in sanity_check_data['random']]

In [ ]:
real_quests_predictions

[{'order': [6, 8, 5, 7, 3, 2, 4, 0, 1], 'score': -7.363732216211075},
 {'order': [0, 1, 2, 3], 'score': -7.802377976082989},
 {'order': [3, 4, 13, 17, 16, 14, 10, 15, 11, 2, 1, 8, 9, 12, 7, 6, 0, 5],
  'score': -7.485924463321846},
 {'order': [9, 4, 6, 8, 1, 5, 7, 3, 0, 2], 'score': -6.794129321447051},
 {'order': [4, 7, 10, 0, 14, 2, 13, 15, 8, 1, 11, 5, 9, 6, 3, 16, 12, 18, 17],
  'score': -5.547068385159401}]

In [ ]:
from statistics import mean
mean([sample['score'] for sample in real_quests_predictions])

-6.998646472444473

In [ ]:
mean([sample['score'] for sample in random_quests_predictions])

-7.3518664475229984

In [ ]:
FULL_LIMIT = GPT3_LIMIT

results = []
fails = []
for file_name in tqdm(os.listdir(NEW_GRAPHS_DIRECTORY)):
    used_current_file_paths = 0
    graph_file = os.path.join(NEW_GRAPHS_DIRECTORY, file_name)
    with open(graph_file) as f:
        G = json_graph.node_link_graph(json.load(f))
        nodes = G.nodes()

    paths_file = os.path.join(NEW_PATHS_DIRECTORY, file_name)
    with open(paths_file) as f:
        paths = json.load(f)
    
    for path in paths:
        corrected_path = [node_id for node_id in path if 'fragment_text' in nodes[node_id] and isinstance(nodes[node_id]['fragment_text'], str)]
        if len(corrected_path) > 3 and len(corrected_path) <= 25:
            try:
                predicted = gpt_beam_search_last_score(corrected_path, G)

                results.append({'correct':path, 'predicted':predicted})
                used_current_file_paths += 1
                if len(results) % 10 == 0:
                    with open('gpt3_ordering_2048_last.json', 'w') as f:
                        json.dump(results, f)
                if used_current_file_paths == 10:
                    break
            except Exception as e:
                fails.append[(file_name, e)]
                
with open('gpt3_ordering_2048_last.json', 'w') as f:
    json.dump(results, f)

100%|██████████| 34/34 [2:40:13<00:00, 282.75s/it]


In [ ]:
with open('gpt3_ordering_2048_last.json') as f:
   results = json.load(f)


correct_first = 0
for result in results:
    if result['predicted'][0] == result['correct'][0]:
        correct_first += 1
correct_first/len(results)

0.058823529411764705

In [ ]:
with open('gpt3_ordering_2048_last.json') as f:
   results = json.load(f)

prelast = 0
last = 0
second = 0
for result in results:
    
    if result['predicted'][-1] == result['correct'][-1]:
        last += 1
    if result['predicted'][-2] == result['correct'][-2]:
        prelast += 1
    if result['predicted'][1] == result['correct'][1]:
        second += 1
print(last/len(results))
print(prelast/len(results))
print(second/len(results))

0.08597285067873303
0.11312217194570136
0.08144796380090498


In [ ]:
gpt_2048_df = pd.DataFrame(columns=['Fragment_length', 'Tau', 'Max correct sequence'])
exceptions = []

with open('gpt3_ordering_2048_last.json') as f:
   results = json.load(f)

for sample in results:
    correct = sample['correct']
    predicted = fix_prediction(sample['predicted'], correct)
    tau = kendalltau(predicted, correct).correlation
    lcs = longest_correct_subsequence(predicted, correct)
    gpt_2048_df.loc[len(gpt_2048_df)] = [len(correct), tau, lcs]

In [ ]:
aggr_gpt2048 = gpt_2048_df.groupby('Fragment_length').describe()[[('Tau',  'count'), ('Tau',  'mean'),  ('Max correct sequence',  'mean')]]
aggr_gpt2048

Tau           Max correct sequence
                count      mean                 mean
Fragment_length                                     
4.0               3.0 -0.555556             1.666667
5.0              15.0  0.093333             1.000000
6.0               7.0 -0.180952             1.285714
7.0               6.0 -0.126984             1.833333
8.0               9.0 -0.039683             2.333333
9.0              16.0 -0.031250             1.812500
10.0             23.0  0.109179             2.130435
11.0             19.0  0.087081             2.105263
12.0             32.0  0.103220             1.468750
13.0             21.0  0.101343             2.380952
14.0             13.0  0.019442             1.692308
15.0             11.0  0.007792             1.909091
16.0              7.0 -0.078571             1.857143
17.0              6.0  0.058824             1.333333
18.0              8.0  0.117647             2.125000
19.0              6.0 -0.087719             2.333333
20.0              5.0 -0.021053             2.200000
21.0              1.0 -0.019048             3.000000
22.0              2.0  0.060606             2.000000
23.0              2.0  0.169960             2.000000
24.0              2.0 -0.010870             1.000000
25.0              5.0  0.088000             2.200000
26.0              2.0  0.163077             2.000000

In [ ]:
with open('latex.txt', 'w') as f:
  f.write(aggr_gpt2048.to_latex())

In [ ]:
class Fragment4Bidirectional:
    bof_token = 'BOF'
    button_token = 'BUTTON'
    input_token = 'INPUT'
    default_input='Далее'

    def __init__(self, fragment_id, graph=None, full_text = None):
        self.fragment_id = fragment_id
        if graph:
            fragment_text = graph.nodes()[fragment_id]['fragment_text']
            edges = graph.edges()
            buttons =  [' '.join([self.button_token, edges[edge]['command']] + [self.default_input]) for edge in graph.out_edges(fragment_id)]
            fragment_text_without_options = ' '.join([self.bof_token, fragment_text]) 
            if buttons:
                fragment_text = fragment_text_without_options + ' '.join(buttons)
        fragment_text = ' '.join([fragment_text, self.input_token, self.default_input])
        self.tokens = tokenizer.encode(fragment_text)
        self.tokens_without_options = tokenizer.encode(fragment_text_without_options)



In [ ]:
def gpt_beam_search_last_score_bidirectional(test_sample, quest_graph, num_best_candidates=2):
    fragments = [Fragment4Bidirectional(node_id, quest_graph) for node_id in test_sample]
    
    order_candidates = [{
        'beginning':[], 
        'ending':[],
        'next_candidates': fragments, 
        'probability':1, 
        'history_start':tokenizer.encode(start_token),
        'history_end': tokenizer.encode(end_token)}]
    random.shuffle(order_candidates[0]['next_candidates'])
    
    for i in range(len(fragments)):
        current_iteration_next_candidates = []
        
        for order_candidate in order_candidates:

            inverted_direction = i%2 == 1

            history_start = order_candidate ['history_start']
            history_end = order_candidate ['history_end']
            
            if inverted_direction:
                 if i >= 2:
                     scored_next = [{'next_fragment':next_fragment, 'score':calc_perplexity_based_score(next_fragment.tokens, history_end)} for next_fragment in order_candidate['next_candidates']]
                 else:
                    scored_next = [{'next_fragment':next_fragment, 'score':calc_perplexity_based_score(next_fragment.tokens_without_options, history_end)} for next_fragment in order_candidate['next_candidates']]
            else:
                scored_next = [{'next_fragment':next_fragment, 'score':calc_perplexity_based_score(history_start, next_fragment.tokens)} for next_fragment in order_candidate['next_candidates']]
            scored_next.sort(key=lambda x: x['score'], reverse=True)
            
            chosen_next_candidates = scored_next[:num_best_candidates] if len(scored_next) < num_best_candidates else scored_next
            
            for chosen_next_candidate in chosen_next_candidates:

                if inverted_direction:
                    next_step_beginning = order_candidate['beginning']
                    next_step_ending = [chosen_next_candidate['next_fragment']] + order_candidate['ending']
                else:
                   next_step_beginning = copy(order_candidate['beginning'])
                   next_step_beginning.append(chosen_next_candidate['next_fragment'])
                   next_step_ending = order_candidate['ending']
                
                next_step_probability = chosen_next_candidate['score']
                
                next_step_next_candidates = []
                for next_step_next_candidate in order_candidate['next_candidates']:
                    if next_step_next_candidate.fragment_id != chosen_next_candidate['next_fragment'].fragment_id:
                        next_step_next_candidates.append(next_step_next_candidate)

                if inverted_direction:
                    next_step_history_starty = history_start
                    if i >= 2:
                        next_step_history_end = chosen_next_candidate['next_fragment'].tokens + history_end
                    else:
                        next_step_history_end = chosen_next_candidate['next_fragment'].tokens_without_options + history_end
                    
                else:
                    next_step_history_start = history_start + chosen_next_candidate['next_fragment'].tokens
                    next_step_history_end = history_end
                current_iteration_next_candidates.append({'beginning':next_step_beginning, 'ending':next_step_ending,'next_candidates':next_step_next_candidates, 'probability':next_step_probability, 'history_start': next_step_history_start, 'history_end': next_step_history_end})
        current_iteration_next_candidates.sort(key=lambda x: x['probability'], reverse=True)
       # print(len(current_iteration_next_candidates))
        if current_iteration_next_candidates:
            order_candidates = current_iteration_next_candidates[:num_best_candidates] if len(current_iteration_next_candidates) > num_best_candidates else  current_iteration_next_candidates
    assert all([not i['next_candidates'] for i in order_candidates])
    for order_candidate in order_candidates:
        order_candidate['probability'] = calc_perplexity_based_score(order_candidate['history_start'], order_candidate['history_end'])
    order_candidates.sort(key = lambda x: x['probability'], reverse=True)
    return [fragment.fragment_id for fragment in order_candidates[0]['beginning']] + [fragment.fragment_id for fragment in order_candidates[0]['ending']]

In [ ]:
FULL_LIMIT = GPT3_LIMIT
start_token = 'BOQ'
end_token = 'EOQ'

results = []
fails = []
for file_name in tqdm(os.listdir(NEW_GRAPHS_DIRECTORY)):
    used_current_file_paths = 0
    graph_file = os.path.join(NEW_GRAPHS_DIRECTORY, file_name)
    with open(graph_file) as f:
        G = json_graph.node_link_graph(json.load(f))
        nodes = G.nodes()

    paths_file = os.path.join(NEW_PATHS_DIRECTORY, file_name)
    with open(paths_file) as f:
        paths = json.load(f)
    
    for path in paths:

        try:
            corrected_path = [node_id for node_id in path if 'fragment_text' in nodes[node_id] and isinstance(nodes[node_id]['fragment_text'], str)]
            if len(corrected_path) > 3 and len(corrected_path) <= 25:
                predicted = gpt_beam_search_last_score_bidirectional(corrected_path, G)

                results.append({'correct':path, 'predicted':predicted})
                used_current_file_paths += 1
                if len(results) % 10 == 0:
                    with open('gpt3_ordering_2048_last_bidirectional.json', 'w') as f:
                        json.dump(results, f)
              
        except Exception as e:
            print(e)
         
                
with open('gpt3_ordering_2048_last_bidirectional.json', 'w') as f:
    json.dump(results, f)

In [ ]:
gpt_2048_df = pd.DataFrame(columns=['Fragment_length', 'Tau', 'Max correct sequence'])

with open('gpt3_ordering_2048_last_bidirectional.json') as f:
   results = json.load(f)

for sample in results:
    correct = sample['correct']
    predicted = sample['predicted']
    tau = kendalltau(predicted, correct).correlation
    lcs = longest_correct_subsequence(predicted, correct)
    gpt_2048_df.loc[len(gpt_2048_df)] = [len(correct), tau, lcs]

In [ ]:
aggr_gpt2048 = gpt_2048_df.groupby('Fragment_length').describe()[[('Tau',  'count'), ('Tau',  'mean'),  ('Max correct sequence',  'mean')]]
aggr_gpt2048

Tau           Max correct sequence
                count      mean                 mean
Fragment_length                                     
4.0               2.0  0.333333             0.000000
5.0               1.0 -0.200000             0.000000
6.0               2.0 -0.133333             1.500000
7.0               6.0  0.047619             1.500000
8.0               9.0  0.031746             1.000000
9.0              25.0  0.084444             1.480000
10.0             28.0  0.146032             1.250000
11.0             21.0  0.139394             1.571429
12.0             56.0 -0.023268             1.267857
13.0             40.0  0.014744             1.975000
14.0             31.0  0.050691             1.806452
15.0             27.0 -0.068783             1.703704
16.0             35.0  0.007143             1.657143
17.0             30.0  0.055882             1.600000
18.0             30.0  0.001743             1.266667
19.0             20.0 -0.074269             1.850000
20.0             26.0  0.021053             1.692308
21.0              6.0 -0.074603             2.166667
22.0              5.0  0.151515             1.800000
23.0              6.0 -0.133070             1.333333
24.0              1.0  0.014493             2.000000
25.0              8.0  0.016667             1.750000

In [ ]:
with open('bidirectional.txt', 'w') as f:
  f.write(aggr_gpt2048.to_latex())

In [ ]:
with open('gpt3_ordering_2048_last_bidirectional.json') as f:
   results = json.load(f)


correct_first = 0
correct_last = 0
for result in results:
    if result['predicted'][0] == result['correct'][0]:
        correct_first += 1
    if result['predicted'][-1] == result['correct'][-1]:
        correct_last += 1
print(correct_first/len(results))
print(correct_last/len(results))

0.025842696629213482
0.15842696629213482


In [ ]:
prelast = 0
second = 0
for result in results:
    if result['predicted'][-2] == result['correct'][-2]:
        prelast += 1
    if result['predicted'][1] == result['correct'][1]:
        second += 1
print(prelast/len(results))
print(second/len(results))

0.12247191011235956
0.16067415730337078
